In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 1. 링크 리스트 불러오기 및 정리
df = pd.read_csv('raw/list_output.csv', header=None)
link_set = [link.strip() for link in df[0].tolist()]  # 공백 제거!

print(f"총 링크 개수: {len(link_set)}")
print(link_set[:5])  # 앞 5개 미리보기

# 2. 결과 저장 리스트
results = []

# 3. User-Agent 고정 (네이버 차단 방지)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

# 4. 크롤링 루프 (테스트로 10개만)
for idx, link in enumerate(link_set[0:1960], 1):  
    url = link
    print(f"\n===== [{idx}] 링크 크롤링 시작 =====\n링크: {url}")

    try:
        response = requests.get(url, headers=headers, timeout=10)
        print(f"[{idx}] 응답 상태 코드: {response.status_code}")

        if response.status_code != 200:
            print(f"[{idx}] 상태코드 오류: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # 질문 가져오기
        question_elements = soup.select('.questionDetail')
        question = question_elements[0].text.strip() if question_elements else '질문 없음'

        # 답변 가져오기
        a = soup.find_all(class_='se-main-container')
        answers = [i.text.strip() for i in a]
        answer_text = ' '.join(answers) if answers else '답변 없음'

        # 결과 추가
        results.append({
            'question': question,
            'answer': answer_text
        })

        print(f"[{idx}] 크롤링 성공! 현재 수집된 데이터 수: {len(results)}")

    except Exception as e:
        print(f"[{idx}] 예외 발생: {e}")

    time.sleep(0.5)  # 서버에 무리 주지 않게 딜레이!

# 5. 결과 저장
result_df = pd.DataFrame(results)
output_path = 'data/raw/crawled_results_test.csv'
result_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f'\n✅ 크롤링 완료! 결과 파일 저장됨 → {output_path}')


총 링크 개수: 1960
['http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40201&docId=482506450', 'http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40204&docId=481823128', 'http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40204&docId=481669847', 'http://kin.naver.com/qna/detail.naver?d1id=4&dirId=4020202&docId=482807919', 'http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40204&docId=481680019']

===== [1] 링크 크롤링 시작 =====
링크: http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40201&docId=482506450
[1] 응답 상태 코드: 200
[1] 크롤링 성공! 현재 수집된 데이터 수: 1

===== [2] 링크 크롤링 시작 =====
링크: http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40204&docId=481823128
[2] 응답 상태 코드: 200
[2] 크롤링 성공! 현재 수집된 데이터 수: 2

===== [3] 링크 크롤링 시작 =====
링크: http://kin.naver.com/qna/detail.naver?d1id=4&dirId=40204&docId=481669847
[3] 응답 상태 코드: 200
[3] 크롤링 성공! 현재 수집된 데이터 수: 3

===== [4] 링크 크롤링 시작 =====
링크: http://kin.naver.com/qna/detail.naver?d1id=4&dirId=4020202&docId=482807919
[4] 응답 상태 코드: 200
[4] 크롤링 성공! 현재 수집된 데이터 수: 4



In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# 파일 불러오기
df = pd.read_csv('raw/crawled_results_test.csv')

# 링크 삭제
url_pattern = r'http[s]?://\S+'
def remove_links(text):
    if pd.isna(text):
        return text
    return re.sub(url_pattern, '', text)

# HTML 태그 제거
def remove_html(text):
    if pd.isna(text):
        return text
    return BeautifulSoup(text, "html.parser").get_text()

# 공백 및 개행 정리
def clean_whitespace(text):
    if pd.isna(text):
        return text
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 전화번호 제거
def remove_phone_numbers(text):
    if pd.isna(text):
        return text
    return re.sub(r'\d{2,4}-\d{3,4}-\d{4}', '', text)

# 이모지 및 특수문자 제거
def remove_emoji_special(text):
    if pd.isna(text):
        return text
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# ✅ ⭐ ● ▶ ※ 등 심볼 제거
def remove_symbols(text):
    if pd.isna(text):
        return text
    symbols_pattern = r'[✅⭐●▶※◎□■▲▽→←↑↓]'
    text = re.sub(symbols_pattern, '', text)
    return text

# 반복 문구 제거
unwanted_phrases = ['상담은', '문의주세요', '좋은 하루', '대표전화']
def remove_phrases(text):
    if pd.isna(text):
        return text
    for phrase in unwanted_phrases:
        text = text.replace(phrase, '')
    return text

# 전처리 순서 적용
for col in ['question', 'answer']:
    df[col] = df[col].apply(remove_links)
    df[col] = df[col].apply(remove_html)
    df[col] = df[col].apply(remove_phone_numbers)
    df[col] = df[col].apply(remove_emoji_special)
    df[col] = df[col].apply(remove_symbols)
    df[col] = df[col].apply(remove_phrases)
    df[col] = df[col].apply(clean_whitespace)

# 너무 짧은 질문/답변 삭제
df = df[(df['question'].str.len() > 10) & (df['answer'].str.len() > 10)]

# 저장
output_path = 'processed/crawled_results_cleaned_full.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f'\n✅ 전처리 완료! 링크와 이모지 및 심볼이 제거된 결과가 "{output_path}"에 저장되었습니다.')



✅ 전처리 완료! 링크와 이모지 및 심볼이 제거된 결과가 "processed/crawled_results_cleaned_full.csv"에 저장되었습니다.


/var/folders/0p/p0wk5frj48q0yv4csjy013m40000gn/T/ipykernel_19511/2983110678.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
